# Hog feature testing

En aquest fitxer, ens encarregarem de testejar l'evaluació del model amb la característica individual HOG(Histogram of Oriented Graphics).

## 1. Configuració

Primer ens encarregarem d'importar totes les llibraries necessàries i definir un FeatureTransformer amb els paràmetres corresponents a la nostra feature.

In [1]:
import cv2
import os
import skimage.feature
from FeatureTransformer import FeatureTransformer
from Utils import process_images_in_folder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import numpy as np

In [2]:
transformer = FeatureTransformer()
hog_params = {
    'orientations': 9,
    'pixels_per_cell': (8, 8),
    'cells_per_block': (2, 2),
    'block_norm': 'L2-Hys',
    'visualize': False,
    'transform_sqrt': True,
    'feature_vector': True
}
customFeature=[skimage.feature.hog,hog_params]

## 2. Feature extraction

In this part, we'll define our variables needed for training and evaluation. We'll also start extracting features from the images.

In [3]:
x_train=[]
y_train=[]
x_test=[]
y_test=[]

In [ ]:
train_folder_path = 'data/train'
test_folder_path = 'data/test'

# Extract features from training data
print('*********************TRAINING*********************')
for class_folder in os.listdir(train_folder_path):
    print("Extraguent features de la classe: " + str(class_folder))
    class_folder_path = os.path.join(train_folder_path, class_folder)
    process_images_in_folder(transformer,customFeature,class_folder_path,class_folder,x_train,y_train)

# Extract features from testing data
print('*********************TESTING*********************')
for class_folder in os.listdir(test_folder_path):
    print("Extraguent features de la classe: " + str(class_folder))
    class_folder_path = os.path.join(test_folder_path, class_folder)
    process_images_in_folder(transformer,customFeature,class_folder_path,class_folder,x_test,y_test)

*********************TRAINING*********************
Extraguent features de la classe: bedroom
Extraguent features de la classe: Coast
Extraguent features de la classe: Forest
Extraguent features de la classe: Highway
Extraguent features de la classe: industrial
Extraguent features de la classe: Insidecity
Extraguent features de la classe: kitchen
Extraguent features de la classe: livingroom
Extraguent features de la classe: Mountain
Extraguent features de la classe: Office
Extraguent features de la classe: OpenCountry
Extraguent features de la classe: store
Extraguent features de la classe: Street
Extraguent features de la classe: Suburb


Finalment, ens encarregam de que estiguin en el format correcte per a passar al model.

In [ ]:
x_train = np.array(x_train).reshape(len(x_train), -1)
x_test = np.array(x_test).reshape(len(x_test), -1)

## 3. Fine Tuning

En aquest apartat, ens encarregarem de trobar les millors configuracions possibles per al nostre model amb el mètode GridSearchCV de Sklearn. Escollirem una llista de paràmetres amb diferents valors i imprimirem per pantalla el millor estimador.

In [ ]:
param_grid = {'C': [0.1,1, 10, 100],'kernel': ['rbf', 'poly', 'sigmoid']}
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=2)
grid.fit(x_train,y_train)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


In [36]:
print(grid.best_estimator_)

SVC(C=1, kernel='poly')


## 4. Training model

Now, having an optimized config for the model, we should be able to train it and get a good result.

In [11]:
model = SVC(C=1.0, kernel='poly', random_state=33)
model.fit(x_train,y_train)

SVC(kernel='poly', random_state=33)

## 5. Evaluating model

Finally, we'll predict our defined data for testing and compare with real expected values. Our goal is to obtain a % for accuracy.

In [12]:
y_meu = model.predict(x_test)

In [10]:
precision_meu = accuracy_score(y_test, y_meu)
print(f"   Accuracy   : {precision_meu}")

NameError: name 'y_meu' is not defined